<a href="https://colab.research.google.com/github/gk0434/Gautam_DTSC_Fall2025/blob/main/Assignment5_Ch_10%2611.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment5: CRM Cleanup @ **DalaShop**
*Files (Ch.10), Exceptions (Ch.10), Unit Tests (Ch.11), and Regular Expressions*  
.....

**Total: 3 points**  (Two questions, 1.5 pts each)  

> This assignment is scenario-based and aligned with Python Crash Course Ch.10 (files & exceptions), Ch.11 (unit testing with `unittest`), and Regular Expressions.

## Scenario
You are a data intern at an online retailer called **DalaShop**.  
Sales exported a **raw contacts** file from the CRM. It contains customer names, emails, and phone numbers, but the formatting is messy and some emails are invalid.  
Your tasks:

1. **Clean** the contacts (Files + Exceptions + Regex).  
2. **Write unit tests** to make sure your helper functions work correctly and keep working in the future.

## Data file (given by the company): `contacts_raw.txt`
Use this exact sample data (you may extend it for your own testing, but do **not** change it when submitting).  
Run the next cell once to create the file beside your notebook.

In [ ]:
# Create the provided company dataset file
with open("contacts_raw.txt", "w", encoding="utf-8") as f:
    f.write('Alice Johnson <alice@example.com> , +1 (469) 555-1234\nBob Roberts <bob[at]example.com> , 972-555-777\nSara M. , sara@mail.co , 214 555 8888\n"Mehdi A." <mehdi.ay@example.org> , (469)555-9999\nDelaram <delaram@example.io>, +1-972-777-2121\nNima <NIMA@example.io> , 972.777.2121\nduplicate <Alice@Example.com> , 469 555 1234')
print("Wrote contacts_raw.txt with sample DalaShop data.")

## Q1 (1.5 pts) — CRM cleanup with Files, Exceptions, and Regex
Implement `q1_crm_cleanup.py` to:

1. **Read** `contacts_raw.txt` using `pathlib` and `with`. If the file is missing, **handle** it gracefully with `try/except FileNotFoundError` (print a friendly message; do not crash).
2. **Validate emails** with a simple regex (`r"[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}"`).  
   - Trim whitespace with `strip()` before checking.  
   - Use **full** matching (not partial).
3. **Normalize phone numbers:** remove all non-digits (e.g., with `re.sub(r"\D", "", raw)`).  
   - If the result has **≥ 10 digits**, keep the **last 10 digits**.  
   - Otherwise, return an **empty string** (`""`).
4. **Filter rows:** keep **only** rows with a valid email.
5. **Deduplicate:** remove duplicates by **email** using **case-insensitive** comparison (e.g., `email.casefold()`). **Keep the first occurrence** and drop later duplicates.
6. **Output CSV:** write to `contacts_clean.csv` with **columns exactly** `name,email,phone` (UTF-8).  
7. **Preserve input order:** the order of rows in `contacts_clean.csv` must match the **first appearance** order from the input file. **Do not sort** the rows.

**Grading rubric (1.5 pts):**
- (0.4) File read/write via `pathlib` + graceful `FileNotFoundError` handling  
- (0.5) Correct email regex validation + filtering  
- (0.4) Phone normalization + case-insensitive de-dup (keep first)  
- (0.2) Clean code, clear names, minimal docstrings/comments

In [10]:
import io, sys, unittest
from pathlib import Path

# --- Write q1_crm_cleanup.py ---
crm_code = r"""
from pathlib import Path
import csv
import re
from typing import List, Dict, Iterable

EMAIL_RE = re.compile(r"[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}$")

def is_valid_email(raw: str) -> bool:
    if not raw:
        return False
    raw = raw.strip()
    return EMAIL_RE.fullmatch(raw) is not None

def normalize_phone(raw: str) -> str:
    if raw is None:
        return ""
    digits = re.sub(r"\D", "", raw)
    if len(digits) >= 10:
        return digits[-10:]
    return ""

def parse_contact_line(line: str) -> Dict[str, str]:
    name = ""
    email = ""
    phone = ""
    line = line.strip()
    if "<" in line and ">" in line:
        parts = [p.strip() for p in line.split(",")]
        left = parts[0]
        phone = parts[1].strip() if len(parts) > 1 else ""
        m = re.search(r"<([^>]+)>", left)
        if m:
            email = m.group(1).strip()
        name = left.split("<", 1)[0].strip().strip('"').strip("'")
    else:
        parts = [p.strip() for p in line.split(",")]
        if len(parts) >= 1:
            name = parts[0]
        if len(parts) >= 2:
            email = parts[1]
        if len(parts) >= 3:
            phone = parts[2]
    phone = normalize_phone(phone)
    return {"name": name, "email": email.strip(), "phone": phone}

def dedupe_by_email_casefold(rows: Iterable[Dict[str, str]]) -> List[Dict[str, str]]:
    seen = set()
    deduped = []
    for row in rows:
        key = row["email"].casefold()
        if key not in seen:
            seen.add(key)
            deduped.append(row)
    return deduped

def load_raw_contacts(path: Path) -> List[str]:
    try:
        with path.open("r", encoding="utf-8") as f:
            return f.readlines()
    except FileNotFoundError:
        print(f"[INFO] File not found: {path!s}. Please make sure contacts_raw.txt exists.")
        return []

def clean_contacts(lines: List[str]) -> List[Dict[str, str]]:
    parsed = []
    for line in lines:
        if not line.strip():
            continue
        row = parse_contact_line(line)
        if is_valid_email(row["email"]):
            parsed.append(row)
    return dedupe_by_email_casefold(parsed)

def write_clean_csv(rows: List[Dict[str, str]], out_path: Path) -> None:
    with out_path.open("w", encoding="utf-8", newline="") as f:
        writer = csv.DictWriter(f, fieldnames=["name", "email", "phone"])
        writer.writeheader()
        writer.writerows(rows)

def main() -> None:
    raw_path = Path("contacts_raw.txt")
    out_path = Path("contacts_clean.csv")
    lines = load_raw_contacts(raw_path)
    if not lines:
        return
    cleaned = clean_contacts(lines)
    write_clean_csv(cleaned, out_path)
    print(f"[OK] Wrote {len(cleaned)} cleaned contacts to {out_path!s}.")

if __name__ == "__main__":
    main()
"""
with open("q1_crm_cleanup.py", "w", encoding="utf-8") as f:
    f.write(crm_code)

## Q2 (1.5 pts) — Unit testing with `unittest`
Create tests in `test_crm_cleanup.py` that cover at least:

1. **Email validation**: valid/invalid variations.  
2. **Phone normalization**: parentheses, dashes, spaces, country code; too-short cases.  
3. **Parsing**: from a small multi-line string (not from a file), assert the exact structured rows (name/email/phone).  
4. **De-duplication**: demonstrate that a case-variant duplicate email is dropped (first occurrence kept).




In [12]:
# --- Write test_crm_cleanup.py ---
test_code = r"""
import unittest
from q1_crm_cleanup import (
    is_valid_email,
    normalize_phone,
    parse_contact_line,
    clean_contacts,
    dedupe_by_email_casefold,
)

class TestCRMCleanup(unittest.TestCase):
    def test_valid_emails(self):
        self.assertTrue(is_valid_email("alice@example.com"))
        self.assertTrue(is_valid_email("user.name+tag@sub.domain.co"))
        self.assertTrue(is_valid_email("  spaced@example.org  "))

    def test_invalid_emails(self):
        self.assertFalse(is_valid_email("bob[at]example.com"))
        self.assertFalse(is_valid_email("no-at-symbol.com"))
        self.assertFalse(is_valid_email("missing-tld@example"))
        self.assertFalse(is_valid_email(""))

    def test_phone_normalization(self):
        self.assertEqual(normalize_phone("+1 (469) 555-1234"), "4695551234")
        self.assertEqual(normalize_phone("214 555 8888"), "2145558888")
        self.assertEqual(normalize_phone("(469)555-9999"), "4695559999")
        self.assertEqual(normalize_phone("972.777.2121"), "9727772121")

    def test_phone_too_short(self):
        self.assertEqual(normalize_phone("555-123"), "")

    def test_phone_long(self):
        self.assertEqual(normalize_phone("+1-972-777-2121"), "9727772121")

    def test_parsing_multiline_string(self):
        data = '''Alice Johnson <alice@example.com> , +1 (469) 555-1234
Sara M. , sara@mail.co , 214 555 8888
"Mehdi A." <mehdi.ay@example.org> , (469)555-9999
'''
        lines = data.splitlines()
        cleaned = clean_contacts(lines)
        self.assertEqual(len(cleaned), 3)
        self.assertEqual(cleaned[0]["email"], "alice@example.com")
        self.assertEqual(cleaned[0]["phone"], "4695551234")
        self.assertEqual(cleaned[2]["name"], "Mehdi A.")

    def test_deduplication_casefold(self):
        rows = [
            {"name": "Alice", "email": "alice@example.com", "phone": "1"},
            {"name": "duplicate", "email": "Alice@Example.com", "phone": "2"},
            {"name": "Bob", "email": "bob@example.com", "phone": "3"},
        ]
        deduped = dedupe_by_email_casefold(rows)
        self.assertEqual(len(deduped), 2)
        self.assertEqual(deduped[0]["name"], "Alice")
        self.assertEqual(deduped[1]["name"], "Bob")

if __name__ == "__main__":
    unittest.main()
"""
with open("test_crm_cleanup.py", "w", encoding="utf-8") as f:
    f.write(test_code)


#Run all unit test
!python3 -m unittest -v test_crm_cleanup.py

test_deduplication_casefold (test_crm_cleanup.TestCRMCleanup.test_deduplication_casefold) ... ok
test_invalid_emails (test_crm_cleanup.TestCRMCleanup.test_invalid_emails) ... ok
test_parsing_multiline_string (test_crm_cleanup.TestCRMCleanup.test_parsing_multiline_string) ... ok
test_phone_long (test_crm_cleanup.TestCRMCleanup.test_phone_long) ... ok
test_phone_normalization (test_crm_cleanup.TestCRMCleanup.test_phone_normalization) ... ok
test_phone_too_short (test_crm_cleanup.TestCRMCleanup.test_phone_too_short) ... ok
test_valid_emails (test_crm_cleanup.TestCRMCleanup.test_valid_emails) ... ok

----------------------------------------------------------------------
Ran 7 tests in 0.001s

OK


## Grading rubric (total 3 pts)
- **Q1 (1.5 pts)**  
  - (0.4) File I/O with `pathlib` + graceful `FileNotFoundError` handling  
  - (0.5) Email validation (regex + strip + full match) and filtering  
  - (0.4) Phone normalization and **case-insensitive** de-duplication (keep first)  
  - (0.2) Code clarity (names, minimal docstrings/comments)
- **Q2 (1.5 pts)**  
  - (0.6) Meaningful coverage for email/phone functions (valid & invalid)  
  - (0.6) Parsing & de-dup tests that assert exact expected rows  
  - (0.3) Standard `unittest` structure and readable test names
